# Using Keras model as Tensorflow Graph

While Keras uses Tensorflow as its backend, the model with weight trained on Keras can be transformed to Tensorflow Graph to be used from Tensorflow.

## Train Keras model for Cifar10 CNN

In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
kerasBKED = os.environ["KERAS_BACKEND"] 
print(kerasBKED)

tensorflow


In [2]:
import keras
from keras.models import load_model
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, InputLayer
from keras.callbacks import EarlyStopping, ModelCheckpoint
import os
import pickle
import numpy as np

Using TensorFlow backend.


In [3]:
# set parameters

batch_size = 32
num_classes = 10
epochs = 100
saveDir = "./cifar10/"
if not os.path.isdir(saveDir):
    os.makedirs(saveDir)

In [4]:
# get dataset

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [5]:
# define Keras model

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [6]:
# optimization and compile

opt = keras.optimizers.adam()

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [7]:
# load pre-trained model

model.load_weights(os.path.join(saveDir, "Cifar10_convert.08-0.58-0.64.hdf5"))

In [8]:
# callbacks

es_cb = EarlyStopping(monitor='val_loss', patience=2, verbose=1, mode='auto')
chkpt = os.path.join(saveDir, 'Cifar10_convert.{epoch:02d}-{loss:.2f}-{val_loss:.2f}.hdf5')
cp_cb = ModelCheckpoint(filepath = chkpt, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

In [9]:
# train

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=[es_cb, cp_cb],
                    shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 292s - loss: 0.5740 - acc: 0.7982 - val_loss: 0.6484 - val_acc: 0.7816
Epoch 2/100
50000/50000 [==============================] - 293s - loss: 0.5564 - acc: 0.8054 - val_loss: 0.6650 - val_acc: 0.7795
Epoch 3/100
50000/50000 [==============================] - 294s - loss: 0.5465 - acc: 0.8087 - val_loss: 0.6646 - val_acc: 0.7794
Epoch 4/100
50000/50000 [==============================] - 294s - loss: 0.5358 - acc: 0.8124 - val_loss: 0.6377 - val_acc: 0.7890
Epoch 5/100
50000/50000 [==============================] - 295s - loss: 0.5226 - acc: 0.8157 - val_loss: 0.6605 - val_acc: 0.7840
Epoch 6/100
50000/50000 [==============================] - 295s - loss: 0.5118 - acc: 0.8210 - val_loss: 0.6422 - val_acc: 0.7948
Epoch 7/100
50000/50000 [==============================] - 295s - loss: 0.5020 - acc: 0.8247 - val_loss: 0.6353 - val_acc: 0.7898
Epoch 8/100
50000/50000 [===============

In [10]:
# evaluate test data

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.639867679524
Test accuracy: 0.7912


## Use Keras trained model on Tensorflow

In [15]:
# load tensorflow and keras backend

import tensorflow as tf
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from keras import backend as K
ksess = K.get_session()
print(ksess)

In [30]:
# transform keras model to tensorflow graph
# the output will be json-like format

K.set_learning_phase(0)
graph = ksess.graph
kgraph = graph.as_graph_def()
print(kgraph)

node {
  name: "conv2d_1_input"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: -1
        }
        dim {
          size: 32
        }
        dim {
          size: 32
        }
        dim {
          size: 3
        }
      }
    }
  }
}
node {
  name: "conv2d_1/random_uniform/shape"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
          dim {
            size: 4
          }
        }
        tensor_content: "\003\000\000\000\003\000\000\000\003\000\000\000 \000\000\000"
      }
    }
  }
}
node {
  name: "conv2d_1/random_uniform/min"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
        }
  

In [31]:
# define output node

num_output = 1
prefix = "output"
pred = [None]*num_output
outputName = [None]*num_output
for i in range(num_output):
    outputName[i] = prefix + str(i)
    pred[i] = tf.identity(model.get_output_at(i), name=outputName[i])
print('output name: ', outputName)

output name:  ['output0']


In [32]:
# convert variables in the model graph to constants

constant_graph = graph_util.convert_variables_to_constants(ksess, ksess.graph.as_graph_def(), outputName)

INFO:tensorflow:Froze 12 variables.
Converted 12 variables to const ops.


In [33]:
# save the model in .pb and .txt

output_dir = "./"
output_graph_name = "keras2tf.pb"
output_text_name = "keras2tf.txt"
graph_io.write_graph(constant_graph, output_dir, output_graph_name, as_text=False)
graph_io.write_graph(constant_graph, output_dir, output_text_name, as_text=True)
print('saved graph .pb at: {0}\nsaved graph .txt at: {1}'.format(
        os.path.join(output_dir, output_graph_name),
        os.path.join(output_dir, output_text_name)))

saved graph .pb at: ./keras2tf.pb
saved graph .txt at: ./keras2tf.txt


In [22]:
# load the previously saved graph

def load_graph(model_file):
    graph = tf.Graph()
    graph_def = tf.GraphDef()

    with open(model_file, "rb") as f:
        graph_def.ParseFromString(f.read())
    with graph.as_default():
        tf.import_graph_def(graph_def)
    return graph
tfmodel = load_graph(os.path.join(output_dir, output_graph_name))

In [29]:
# print operations in the model, in tf.Operation format

opers = tfmodel.get_operations()
opers

[<tf.Operation 'import/conv2d_1_input' type=Placeholder>,
 <tf.Operation 'import/conv2d_1/kernel' type=Const>,
 <tf.Operation 'import/conv2d_1/kernel/read' type=Identity>,
 <tf.Operation 'import/conv2d_1/bias' type=Const>,
 <tf.Operation 'import/conv2d_1/bias/read' type=Identity>,
 <tf.Operation 'import/conv2d_1/convolution' type=Conv2D>,
 <tf.Operation 'import/conv2d_1/BiasAdd' type=BiasAdd>,
 <tf.Operation 'import/activation_1/Relu' type=Relu>,
 <tf.Operation 'import/conv2d_2/kernel' type=Const>,
 <tf.Operation 'import/conv2d_2/kernel/read' type=Identity>,
 <tf.Operation 'import/conv2d_2/bias' type=Const>,
 <tf.Operation 'import/conv2d_2/bias/read' type=Identity>,
 <tf.Operation 'import/conv2d_2/convolution' type=Conv2D>,
 <tf.Operation 'import/conv2d_2/BiasAdd' type=BiasAdd>,
 <tf.Operation 'import/activation_2/Relu' type=Relu>,
 <tf.Operation 'import/max_pooling2d_1/MaxPool' type=MaxPool>,
 <tf.Operation 'import/dropout_1/keras_learning_phase' type=Placeholder>,
 <tf.Operation 'imp

In [24]:
# define the input layer, learning phase and output layer

inLayer = tfmodel.get_operation_by_name('import/conv2d_1_input')
learnPhase = tfmodel.get_operation_by_name('import/dropout_1/keras_learning_phase')
outLayer = tfmodel.get_operation_by_name('import/output0')

In [25]:
# use the model with test data to predict the label

with tf.Session(graph=tfmodel) as sess:
    results = sess.run(outLayer.outputs[0], 
                       {inLayer.outputs[0]: x_test,
                        learnPhase.outputs[0]: 0})
print(results)

[[  2.96443091e-06   3.40904478e-07   1.25055021e-05 ...,   1.26586031e-06
    3.93378514e-06   8.05182935e-06]
 [  2.26680844e-04   1.04790370e-05   4.63014772e-14 ...,   2.69345000e-19
    9.99762833e-01   2.27002914e-08]
 [  6.96042776e-02   5.21890819e-03   3.25239962e-05 ...,   1.52289849e-05
    9.23621774e-01   1.15730893e-03]
 ..., 
 [  2.73814976e-07   2.31757316e-08   2.28706957e-03 ...,   3.97272408e-03
    1.48406736e-08   7.29675520e-09]
 [  3.05041391e-03   9.95513618e-01   5.43668248e-05 ...,   3.05739784e-04
    1.91562390e-06   2.53295118e-04]
 [  2.90193425e-09   5.23458664e-13   3.33991494e-07 ...,   9.95446205e-01
    5.45167340e-12   1.55371271e-11]]


## References
- https://blog.keras.io/keras-as-a-simplified-interface-to-tensorflow-tutorial.html
- https://github.com/amir-abdi/keras_to_tensorflow
- https://github.com/bitbionic/keras-to-tensorflow